In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef,recall_score, precision_score
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split

In [9]:
batch_size = 128
loaded_datasets_info = torch.load('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Data/saved_datasets.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
lr_model = LogisticRegression(max_iter=1000, random_state=42)
predictions_val = cross_val_predict(lr_model, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]# five-fold 
AUC_val = roc_auc_score(y_train, predictions_val)
ACC_val = accuracy_score(y_train, (predictions_val > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_train, (predictions_val > 0.5).astype(int))
Sn_val = recall_score(y_train, (predictions_val > 0.5).astype(int))
Sp_val = precision_score(y_train, (predictions_val > 0.5).astype(int))
F1_val = f1_score(y_train, (predictions_val > 0.5).astype(int))
lr_model.fit(X_train, y_train)
predictions_test = lr_model.predict_proba(X_test)[:, 1]
AUC_test = roc_auc_score(y_test, predictions_test)
ACC_test = accuracy_score(y_test, (predictions_test > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (predictions_test > 0.5).astype(int))
Sn_test = recall_score(y_test, (predictions_test > 0.5).astype(int))
Sp_test = precision_score(y_test, (predictions_test > 0.5).astype(int))
F1_test = f1_score(y_test, (predictions_test > 0.5).astype(int))
print("Validation Metrics:")
print("AUC_val: {:.4f}".format(AUC_val))
print("ACC_val: {:.4f}".format(ACC_val))
print("MCC_val: {:.4f}".format(MCC_val))
print("Sn_val: {:.4f}".format(Sn_val))
print("Sp_val: {:.4f}".format(Sp_val))
print("F1_val: {:.4f}".format(F1_val))

print("Test Metrics:")
print("AUC_test: {:.4f}".format(AUC_test))
print("ACC_test: {:.4f}".format(ACC_test))
print("MCC_test: {:.4f}".format(MCC_test))
print("Sn_test: {:.4f}".format(Sn_test))
print("Sp_test: {:.4f}".format(Sp_test))
print("F1_test: {:.4f}".format(F1_test))

XGBoost

In [12]:
pos_domain = pd.read_csv('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Data/pos_domain_encoding_35.csv') 
neg_domain = pd.read_csv('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Data/neg_domain_encoding_35.csv') 
pos_sequence = pd.read_csv('D:/Jupyter_directory/Res-F5C-main/Datas/pos_encoding_OH_ND.csv') 
neg_sequence = pd.read_csv('D:/Jupyter_directory/Res-F5C-main/Datas/neg_encoding_OH_ND.csv') 
pos = pd.merge(pos_domain, pos_sequence, left_index=True,right_index=True)
neg = pd.merge(neg_domain, neg_sequence, left_index=True,right_index=True)
datas = np.concatenate((pos,neg),axis = 0)                                 
datas = np.expand_dims(datas, axis = -2)
labels = np.concatenate(([1] * pos.shape[0], [0] * neg.shape[0]),axis = 0)
labels = np.expand_dims(labels,-1)
datas_tensor = torch.tensor(datas, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)
dataset = TensorDataset(datas_tensor, labels_tensor)
train_size = int(0.8 * len(dataset))
train_size_1 = int(0.8 * train_size)
val_size = train_size - train_size_1
train_size = train_size_1
test_size = len(dataset) - train_size - val_size
loaded_train_dataset, loaded_val_dataset, loaded_test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [23]:
loaded_datasets_info = torch.load( 'D:/Jupyter_directory/F5C-codes/Results/Genomic features/XG/saved_datasets.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [24]:
X_train = []
y_train = []
for x, y in loaded_train_dataset:
    X_train.append(x)
    y_train.append(y)
X_test = []
y_test = []
for x, y in loaded_test_dataset:
    X_test.append(x)
    y_test.append(y)
X_val = []
y_val = []
for x, y in loaded_val_dataset:
    X_val.append(x)
    y_val.append(y)

In [26]:
y_train = np.array(torch.stack(y_train)).reshape(-1,)
y_test = np.array(torch.stack(y_test)).reshape(-1,)
y_val = np.array(torch.stack(y_val)).reshape(-1,)
X_train = np.array(torch.stack(X_train)).reshape(-1, 240)
X_test = np.array(torch.stack(X_test)).reshape(-1, 240)
X_val = np.array(torch.stack(X_val)).reshape(-1, 240)

In [29]:
import xgboost as xgb
import matplotlib.pyplot as plt

In [30]:
loaded_model = xgb.XGBRegressor(
    objective='reg:squarederror',  
    learning_rate=0.1,
    max_depth=10,
    n_estimators=30
)
xgb_model.load_model("D:/Jupyter_directory/F5C-codes/Results/Genomic features/XG/xgb_model.xgb")

In [31]:
y_pred = xgb_model.predict(X_test)
AUC_test = roc_auc_score(y_test, y_pred)
ACC_test = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.7850631038141508 0.7014531043593131 0.41628207337413675 0.8218085106382979 0.6602564102564102 0.7322274881516587


In [33]:
y_pred = xgb_model.predict(X_val)
AUC_test = roc_auc_score(y_val, y_pred)
ACC_test = accuracy_score(y_val, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_val, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_val, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_val, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_val, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.7865369514004015 0.6897689768976898 0.3914882338109515 0.7842465753424658 0.6468926553672316 0.7089783281733746


SVM

In [35]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
svm_classifier = SVC(kernel='linear', C=1.0, probability = True)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict_proba(X_test)

In [36]:
y_pred = y_pred[:,1]
AUC_val = roc_auc_score(y_test, y_pred)
ACC_val = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_val = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_val = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_val = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_val, ACC_val, MCC_val, Sn_val, Sp_val, F1_val)

0.7490855531356453 0.6842800528401585 0.3714072005567477 0.7393617021276596 0.6634844868735084 0.69937106918239
